In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sys.path.insert(0, os.path.abspath('../../..'))
from g4l.estimators import BIC
from g4l.estimators import SMC
import g4l.display
from g4l.estimators import Prune
from g4l.models import ContextTree
from g4l.models import integrity
from g4l.data import Sample

## Comparing the mutual ocurrence of trees between methods

In [14]:
results_folder = os.path.abspath('../../example2/results')
methods = ['SeqROCTM', 'prune', 'smc']
models = ['model1']
sizes = [5000]
correct_model = '000 1 10 100'

def get_results(method, model_name, size):
    file = '%s/%s/%s_%s.csv' % (results_folder, method, model_name, size)
    df = pd.read_csv(file)
    df.tree = df.tree.astype(str).map(lambda t: ' '.join(sorted(t.split())))
    return df

def get_champion_trees(df1, df2, sample_idx):
    trees1 = df1[df1.sample_idx == sample_idx]
    trees2 = df2[df2.sample_idx == sample_idx]
    
    #trees2.tree = trees2.tree.astype(str).map(lambda t: ' '.join(sorted(t.split())))
    trees2['match'] = trees2.tree.isin(trees1.tree).astype(int)
    trees1['match'] = trees1.tree.isin(trees2.tree).astype(int)
    return trees1, trees2

def compare(method1, method2, model, size):
    df = pd.DataFrame(columns=['sample_idx', 'precision', 'recall'])
    df1 = get_results(method1, model, size)
    df2 = get_results(method2, model, size)
    for sample_idx in df1.sample_idx.unique():
        trees1, trees2 = get_champion_trees(df1, df2, sample_idx)
        precision = trees2['match'].sum()/len(trees2)
        recall = trees1['match'].sum()/len(trees1)
        df.loc[len(df)] = [int(sample_idx), precision, recall]
    return df, df1, df2

In [15]:
#df, df1, df2 = compare('SeqROCTM', 'smc', 'model1', 5000)
df.mean()

<ipython-input-14-8ad8915a2646>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trees2['match'] = trees2.tree.isin(trees1.tree).astype(int)
<ipython-input-14-8ad8915a2646>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trees1['match'] = trees1.tree.isin(trees2.tree).astype(int)


sample_idx    49.5
precision      1.0
recall         1.0
dtype: float64

In [16]:
#x = get_results('SeqROCTM', 'model1', 5000)
#x.head(30)

In [29]:
method_name = 'smc'
model = 'model1'
size = 5000
#df, df1, df2 = get_results(method_name, model, size)
df, df1, df2 = compare('SeqROCTM', 'smc', 'model1', size)
prec_rec = df[['precision', 'recall']].mean()
dfr = df1[df1.tree=='000 1 10 100']
dfr2 = df2[df2.tree=='000 1 10 100']
correct_tree_occurrence = len(dfr)
found = dfr.opt.sum()
found2 = dfr2.opt.sum()
print("Method:", method_name)
print("Size:", size)
print("Model:", model)
print("Occurrences of correct tree:", correct_tree_occurrence)
print("Found by python version:", found2)
print("Found by matlab version:", found)
print("Precision: ", prec_rec.precision)
print("Recall: ", prec_rec.recall)

<ipython-input-14-8ad8915a2646>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trees2['match'] = trees2.tree.isin(trees1.tree).astype(int)
<ipython-input-14-8ad8915a2646>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trees1['match'] = trees1.tree.isin(trees2.tree).astype(int)


Method: smc
Size: 5000
Model: model1
Occurrences of correct tree: 37
Found by python version: 16
Found by matlab version: 27
Precision:  1.0
Recall:  1.0


In [27]:
dfr

,model_name,sample_idx,method,tree_idx,tree,num_contexts,likelihood,opt
5,model1,0,seqROCTM,5,000 1 10 100,4,-1658.0061,1
34,model1,3,seqROCTM,4,000 1 10 100,4,-1696.2428,0
59,model1,6,seqROCTM,5,000 1 10 100,4,-1669.6051,0
132,model1,13,seqROCTM,5,000 1 10 100,4,-1697.8968,1
145,model1,14,seqROCTM,9,000 1 10 100,4,-1667.1424,1
217,model1,22,seqROCTM,5,000 1 10 100,4,-1718.2828,1
228,model1,23,seqROCTM,7,000 1 10 100,4,-1712.6315,1
256,model1,26,seqROCTM,5,000 1 10 100,4,-1618.2047,1
301,model1,31,seqROCTM,6,000 1 10 100,4,-1703.8961,1
312,model1,32,seqROCTM,7,000 1 10 100,4,-1714.3561,0


'000 1 10 100'